In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pd.options.display.max_rows = 100

cv=pd.read_csv("idk.csv", encoding_errors="replace")

cv=cv.str.replace(" ", "").replace("?", "NaN").astype(float)
#cv["Number_built"] = cv["Number_built"].str.replace(" ", "").astype(float)
#cv["First_flight"]=cv["First_flight"].astype("str").str.replace("?", "NaN").astype(float)

In [ ]:
cv.sort_values(by=["Number_built"], ascending=False).head(50)

In [ ]:


wmap_frame=pd.DataFrame(cv[["Country"]].value_counts().reset_index())
wmap = px.choropleth(wmap_frame, locations="Country", locationmode="country names", color="count", color_continuous_scale="ylorbr", range_color=[0, 500], scope="world")
wmap.update_layout(coloraxis_colorbar=dict(title="No. of models", tickvals=[0, 100, 200, 300, 400, 500], ticktext=["0", "100", "200", "300", "400", ">500"]))
 
wmap.show()


In [ ]:
cv.groupby('Manufacturer')['Number_built'].sum().sort_values(ascending=False)

In [ ]:

nmap_frame=pd.DataFrame(cv.groupby("Country")['Number_built'].sum().sort_values(ascending=False).reset_index())

nmap = px.choropleth(nmap_frame, locations="Country", locationmode="country names", color="Number_built", color_continuous_scale="ylorbr", range_color=[0, 50000], scope="world")
#nmap.update_layout(coloraxis_colorbar=dict(title="No. of models", tickvals=[0, 100, 200, 300, 400, 500], ticktext=["0", "100", "200", "300", "400", ">500"]))
 
nmap.show()


In [ ]:
year_frame=cv[["First_flight", "Manufacturer"]].copy()
year_frame["First_flight"] = (year_frame["First_flight"].str.replace("?", "NaN").astype(float) // 10 * 10).astype("Int64")
year_frame.groupby('First_flight')['Manufacturer'].size().sort_index(ascending=True)

#y_map=px.bar(year_frame.groupby('First_flight')['Manufacturer'].size())
#y_map.update_layout(xaxis=dict(tickvals=[n for n in range(1900, 2020, 10)]))
#y_map.show()

In [ ]:
r_frame=pd.DataFrame({"First_flight" : (cv["First_flight"].str.replace("?", "NaN").astype(float) // 10 * 10).astype("Int64"), "Country" : cv["Country"]})

new_frame=pd.DataFrame(r_frame.groupby("First_flight").value_counts()).reset_index()
new_frame=new_frame.pivot(index="Country", columns="First_flight", values="count").drop("_unknown_", axis=0)

r_frame=r_frame.groupby('First_flight')['Country'].size().sort_index(ascending=True).astype(float).reset_index()


for co in new_frame.columns:
    new_frame[co] = new_frame.where(new_frame.eq(new_frame.sort_values(by=co, ascending=False).head(5)))[co]

new_frame=new_frame.transpose().dropna(axis=1, how="all")
new_frame["Other"] = r_frame["Country"].values - new_frame.sum(axis=1, numeric_only=True).values


px.bar(new_frame, x=new_frame.index, y=new_frame.columns)


In [ ]:
role_frame=pd.DataFrame({"1900" : cv.query("First_flight >= 1900  and First_flight <= 1990").value_counts("Role") , "1991" : cv.query("First_flight >= 1991 ").value_counts("Role")})
role_frame.head(20)

pie=make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]], subplot_titles=["1900 - 1990", "1991-2019"])
pie.add_trace(go.Pie(labels=role_frame.index, values=role_frame["1900"]), 1, 1)
pie.add_trace(go.Pie(labels=role_frame.index, values=role_frame["1991"]), 1, 2)
pie.show()

role_frame["p1900"] = role_frame["1900"] / role_frame["1900"].sum() * 100
role_frame["p1991"] = role_frame["1991"] / role_frame["1991"].sum() * 100
role_frame["Change"] = ((role_frame["p1991"] - role_frame["p1900"]) / role_frame["p1900"]).round(2).fillna(-1)

px.bar(role_frame["Change"])
